In [ ]:
#!/bin/python3

# Import Modules
import os, json
import pandas as pd
import pdfkit as pdf
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import warnings

# Global Settings
root_folder = './dataset/api'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100) 

# Load Data
df_prof = pd.read_json(f"{root_folder}/profile.json")
df_nutri = pd.read_json(f"{root_folder}/nutrition.json")
json_weight = pd.read_json(f"{root_folder}/weight.json")
df_activ = pd.read_json(f"{root_folder}/activity.json")

In [ ]:
import pandas as pd

with open("./dataset/api/nutrition.json", 'r') as file:
  json_nutri = json.load(file)
#df = pd.json_normalize(json_nutri)
#display(df)

def get_entry(json_data, path, key_name): 
  def get_value(data, path_list): 
    for key in path_list: 
      if isinstance(data, list): 
        try: 
          key = int(key) 
          data = data[key] 
        except (IndexError, ValueError): 
          return None
      else: 
        data = data.get(key, None) 
        if data is None: 
          return None 
    return data 
  # Convert the path to a list of keys 
  path_list = path.strip("][").replace("'", "").split("][") 
  # Flatten the JSON data based on the supplied path 
  flattened_data = [] 
  for date, values in json_data.items(): 
    value = get_value(values, path_list) 
    if value is not None: 
      flattened_data.append({'Date': date, f"{key_name}": value}) 
  # Convert the restructured JSON data to a DataFrame 
  df = pd.DataFrame(flattened_data) 
  return df

df_brand = get_entry(json_nutri, "['foods'][0]['loggedFood']['brand']", "Brand")
df_name = get_entry(json_nutri, "['foods'][0]['loggedFood']['name']", "Name")
display(df)


,Date,Brand
0,2024-11-18,Arla
1,2024-11-17,McDonald's
2,2024-11-16,Harvest Morn
3,2024-11-15,Arla
4,2024-11-14,McVitie's
5,2024-11-13,McVitie's
6,2024-11-12,Arla


In [96]:
import pandas as pd

def get_entry(json_data, path, key_name): 
  def get_value(data, path_list): 
    for key in path_list: 
      if isinstance(data, list): 
        try: 
          key = int(key) 
          data = data[key] 
        except (IndexError, ValueError): 
          return None
      else: 
        data = data.get(key, None) 
        if data is None: 
          return None 
    return data 
  # Convert the path to a list of keys 
  path_list = path.strip("][").replace("'", "").split("][") 
  # Flatten the JSON data based on the supplied path 
  flattened_data = [] 
  for date, values in json_data.items(): 
    value = get_value(values, path_list) 
    if value is not None: 
      flattened_data.append({'Date': date, f"{key_name}": value}) 
  # Convert the restructured JSON data to a DataFrame 
  df = pd.DataFrame(flattened_data) 
  df['Date'] = pd.to_datetime(df['Date'])
  return df

def get_nutrition(json_data, keys):
  flattened_data = []
  for date, values in json_data.items():
    for food in values.get('foods', []):
      entry = {'Date': date}
      for key, friendly_name in keys.items():
        path_list = key.strip("][").replace("'", "").split("][")
        data = food
        for p in path_list:
          if isinstance(data, list):
            try:
              p = int(p)
              data = data[p]
            except (IndexError, ValueError):
              data = None
              break
          else:
            data = data.get(p, None)
            if data is None:
              break
        entry[friendly_name] = data
      flattened_data.append(entry)
  df = pd.DataFrame(flattened_data)
  df['Date'] = pd.to_datetime(df['Date'])
  return df

# Define the paths and their friendly names
keys = {
    "['loggedFood']['mealTypeId']": "Meal",
    "['loggedFood']['brand']": "Brand",
    "['loggedFood']['name']": "Name",
    "['loggedFood']['amount']": "Amount",
    "['loggedFood']['unit']['name']": "Unit",
    "['nutritionalValues']['calories']": "Calories",
    "['nutritionalValues']['carbs']": "Carbs",
    "['nutritionalValues']['fat']": "Fat",
    "['nutritionalValues']['protein']": "Protein"
}

with open("./dataset/api/nutrition.json", 'r') as file:
  json_nutri = json.load(file)
with open("./dataset/api/water.json", 'r') as file:
  json_water = json.load(file)

df_water = get_entry(json_water, "['summary']['water']", "Water In (ml)")

df_nutri = get_nutrition(json_nutri, keys)

day_total = df_nutri.groupby('Date').sum(numeric_only=True).reset_index() 
day_total['Meal'] = 'Total' 
day_total['Brand'] = None 
day_total['Name'] = None 
day_total['Unit'] = None 
day_total = pd.merge(day_total, df_water, on='Date', how='left')
df_combined = pd.concat([df_nutri, day_total], ignore_index=True) 
df_combined = df_combined.sort_values(by=['Date', 'Meal']).reset_index(drop=True) 
df_combined = df_combined.fillna('')
display(df_combined)


,Date,Meal,Brand,Name,Amount,Unit,Calories,Carbs,Fat,Protein,Water In (ml)
0,2024-11-12,1,Arla,Protein Strawberry Yogurt,200.0,gram,144,13.20,0.40,20.00,
1,2024-11-12,1,Zombie,Fruit Bar,1.0,bar,82,20.00,0.00,0.00,
2,2024-11-12,1,Nakd,Peanut Butter Protein Bar,1.0,serving,200,17.70,9.90,7.80,
3,2024-11-12,3,Mai Thai,Jasmine Fragrant Rice,75.0,gram,283,63.75,0.75,4.50,
4,2024-11-12,3,Brown Cow,Blueberry Yogurt,480.0,gram,485,57.60,14.40,14.40,
...,...,...,...,...,...,...,...,...,...,...,...
69,2024-11-18,3,,"Mushrooms (with Salt, Drained, Cooked, Boiled)",50.0,gram,14,2.64,0.23,1.09,
70,2024-11-18,3,,Broccoli,70.0,gram,24,4.65,0.26,1.98,
71,2024-11-18,3,John West,Tuna Chunks in Sunflower Oil,132.0,gram,238,0.00,10.56,35.64,
72,2024-11-18,3,Morrisons,Rose Harissa Paste,20.0,gram,28,1.42,2.20,0.40,
